# WORK IN PROGRESS

In [ ]:
from gradflow import Tensor
from gradflow.model import Model
import gradflow.functions as F
from gradflow.optim import *

from datasets.mnist import MNISTDataset

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class Net1(Model):
  def __init__(self):
    super().__init__()

    self.conv1 = F.Conv2d(1, 16, 3)
    # self.bn = F.BatchNorm2d(16)
    self.maxpool1 = F.MaxPool2d(2,2)

    self.dense0 = F.Linear(16*13*13, 10)
    self.relu = F.ReLU()
    self.logsoftmax = F.LogSoftmax(dim=-1)
  
  def forward(self, x) -> Tensor:
    bs, c, h, w = x.shape
    out = self.relu(self.conv1(x))
    # out = self.bn(out)
    out = self.maxpool1(out)
    out = out.reshape(bs, -1)
    out = self.relu(self.dense0(out))
    return self.logsoftmax(out)

# net = Net1()
# net(Tensor(np.random.randn(1, 1, 28, 28)))

In [ ]:
train_dset = MNISTDataset("../data/", batch_size=32*3, shuffle=True, flatten=False)
test_dset = MNISTDataset("../data/", batch_size=10, train=False, flatten=False)

In [ ]:
def dset_loss_acc(dset, model, criterion):
  loss = 0
  corrects = 0
  for imgs, labels in dset:
    outputs = model(imgs)
    loss += criterion(outputs, labels).data

    preds = outputs.data.argmax(axis=1)
    labels = labels.data.argmax(axis=1)
    corrects += ((preds == labels).sum() / labels.size)

  return loss / len(dset), corrects / len(dset)
    

In [ ]:
model = Net1()
criterion = F.NLLLoss(indexed=False)
optimizer = Adam(model.parameters(), 0.001, 0)

In [ ]:
16*13*13

In [ ]:

training_loss = []
corrects = []
for epoch in range(3):
  for i, (imgs, labels) in enumerate(train_dset):
    optimizer.zero_grad()

    outputs = model(imgs)
    loss = criterion(outputs, labels)

    training_loss += [loss.data]
    preds = outputs.data.argmax(axis=1)
    labels = labels.data.argmax(axis=1)
    corrects += [(preds == labels).sum() / labels.size]

    loss.backward()

    optimizer.step()

    if ((i + 1) % 100 == 0):
      test_loss, test_acc = dset_loss_acc(test_dset, model, criterion)
      print("=============================")
      print("Last 100 train | avg. loss: %.4f, acc: %.4f" \
            % (np.mean(training_loss[-100:]), np.mean(corrects[-100:])))
      print("Test           | avg. loss: %.4f, acc: %.4f" % (test_loss, test_acc))
      print("=============================")

In [ ]:
batch = 3

true_imgs, true_labels = test_dset[batch]
labels = true_labels.data.argmax(axis=1)
preds = model(true_imgs)
preds_labels = preds.data.argmax(axis=1)

In [ ]:
true_labels.data

In [ ]:
labels

In [ ]:
preds.data

In [ ]:
preds_labels

In [ ]:
labels

In [ ]:
(preds_labels == labels).astype(np.uint8)